# Correlation and Blocking

One important part of the sampling is to estimate the correlation between the different sampling points.
To this end let's import the following modules

In [ ]:
from qmctorch.scf import Molecule
from qmctorch.wavefunction import SlaterJastrow
from qmctorch.sampler import Metropolis
from qmctorch.solver import Solver
from qmctorch.utils import set_torch_double_precision
from qmctorch.utils import blocking, plot_blocking_energy
from qmctorch.utils import plot_correlation_coefficient, plot_integrated_autocorrelation_time

## Setting up the system

Let's create a simple H2 molecule and a Slater Jastrow wave fuction to demonstrate the correlation properties of the sampling

In [ ]:
set_torch_double_precision()
mol = Molecule(atom = 'H 0. 0. 0; H 0. 0. 1.', unit='bohr', redo_scf=True)

In [ ]:
wf = SlaterJastrow(mol, configs='ground_state')

We can also define a simple Metropolis sampler:

In [ ]:
sampler = Metropolis(nwalkers=100, nstep=500, step_size=0.25,
                    nelec=wf.nelec, ndim=wf.ndim,
                    init=mol.domain('normal'),
                    ntherm=0, ndecor=1,
                    move={'type': 'all-elec', 'proba': 'normal'})

note that by setting `nthemr=0` and `ndecor=1`, we record all the walker positions along the trajectory. 

We can then define the solver:

In [ ]:
solver = Solver(wf=wf, sampler=sampler)

## Correlation coefficient

The simplest way to estimate the decorelation time is to compute the autocorrelation coefficient of the local energy. To this end we must first record the sampling trajectory, i.e. the values of the local energies along the path of the walkers:

In [ ]:
pos = solver.sampler(solver.wf.pdf)
obs = solver.sampling_traj(pos)

We can then plot the correlation coefficient with:

In [ ]:
rho, tau = plot_correlation_coefficient(obs.local_energy)

On this picture is represented the autocorrelation coefficient of the local energy of all the walkers (transparent colorful line)
and the average of the autocorrelation coefficient (thick black line). This mean value is fitted with an exponential decay
to obtain the autocorrelation time that is here equal to 1.79 MCs.

## Integrated autocorrelation time

Another way to estimate the correlation time is to compute the integrated autocorrelation time. This can be done with

In [ ]:
plot_integrated_autocorrelation_time(obs.local_energy)

A conservative estimate of the correlation time can be obtain when the iac cross the dashed line, leading here to a value of about 20 steps.

## Energy blocking

It is also common practice to use blocking of the local energy values to reduce the variance. This can easily be done with

In [ ]:
eb = plot_blocking_energy(obs.local_energy, block_size=100, walkers='mean')

That shows the raw and blocked values of the mean local energy values.